In [1663]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
import re
from itertools import combinations

In [1664]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
311,tt1019452,7000000,31430334,A Serious Man,Michael Stuhlbarg|Richard Kind|Fred Melamed|Sa...,Ethan Coen|Joel Coen,...seriously!,A Serious Man is the story of an ordinary man'...,105,Comedy|Drama,Mike Zoss Productions|Studio Canal|Relativity ...,10/2/2009,6.4,2009
1106,tt0172156,130000000,273339556,Bad Boys II,Martin Lawrence|Will Smith|Jordi MollÃ |Gabrie...,Michael Bay,"If you can't stand the heat, get out of Miami.","Out-of-control, trash-talking buddy cops Marcu...",147,Adventure|Action|Comedy|Thriller|Crime,Columbia Pictures Corporation|Don Simpson/Jerr...,7/18/2003,6.3,2003
1573,tt0167190,66000000,99318987,Hellboy,Ron Perlman|Selma Blair|Rupert Evans|John Hurt...,Guillermo del Toro,From the Dark Side to Our Side.,"In the final days of World War II, the Nazis a...",122,Fantasy|Action|Science Fiction,Revolution Studios|Dark Horse Entertainment|La...,4/2/2004,6.4,2004
594,tt0266391,7000000,3646994,The Cat's Meow,Kirsten Dunst|Edward Herrmann|Eddie Izzard|Car...,Peter Bogdanovich,Some secrets won't stay buried.,Semi-true story of the Hollywood murder that o...,114,Drama|Mystery|Thriller,Lions Gate Films|Dan Films|CP Medien AG,8/3/2001,5.7,2001
1157,tt0279111,56000000,12923936,Gods and Generals,Stephen Lang|Jeff Daniels|Robert Duvall|Kevin ...,Ronald F. Maxwell,The nations heart was touched by...,The film centers mostly around the personal an...,214,Drama|History|War,Turner Pictures|Antietam Filmworks,2/21/2003,5.8,2003


In [1665]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [1730]:
answers = {} # создадим словарь для ответов

# тут другие ваши предобработки колонок например:

#the time given in the dataset is in string format.
#So we need to change this in datetime format
# ...

def preparation_answer(incomming_data):
    ''' Функция для подготовки ответа в формате платформы.
        Формирует выходной результат из имени фильма и его идентификатора'''
    
    # берем первую строку дата фрейма и определяем имя фильма
    movie_name = incomming_data.original_title.iloc[0]
    
    #  определяем id фильма    
    movie_id = incomming_data.imdb_id.iloc[0]
    print(movie_name, ' (', movie_id , ')', sep='')
    return



def year_detect(release_date):
    ''' Функция получения года из четырех цифр, идущих подряд.
        Если год не найден во входной строке, будет возвращено 1970'''
    
    # регулярным выражением проверяем в строке 4 цифры подряд
    result = re.search(r'\d{4}', release_date)
   
    # если найдено
    if result:
        # получаем найденную подстроку и приводим к типу - целое
        return int(result.group(0))
    else:
        # вернем это значение если не смогли найти по шаблону ничего
        return 1970


def month_detect(release_date):
    ''' Функциия получения месяца из первых двух или одно цифры, идущих в начале строки.
        Если цифры не найдены в начале строки - вернет 0. Если полученный номер месяца вне диапазона - вернет 0'''
    
    # регулярным выражением проверяем в строке 2 цифры подряд или 1 цифру в начале строки
    result = re.search(r'^\d\d?', release_date)
    
    # если совпадение есть с шаблоном регулярного выражения
    if result:
        # получаем найденную подстроку и приводим к типу - целое
        month = int(result.group(0))
        # проверяем что значение месяца в валидном диапазоне, иначе возвращем - 0
        if month < 1 or month > 12:
            month = 0
        return month
    else:
        return 0


def series_aggregation_first_column(incoming_data, agg_type='sum', top=5, sort_direction=False,):
    ''' Функция на вход  получвает серию, далее разделяя индексы по символу "|" подсчитывает сумму значений
        Выходной результат - ранжированная серия, ограниченная по длине'''
    
    # создаем пустую серию для выходного результата 
    result_series = pd.Series(dtype=object)
    
    # проходим циклом по парам - индекс и значение 
    for column1, column2 in incoming_data.items():
        # если вид агрегации накопительный, то будем суммировать. Иначе - считать кол-во
        if agg_type == 'sum':
            agg_parametr = column2
        else:
            agg_parametr = 1
        # если индес составной - разделяем на отдельные занчения    
        for item in column1.split('|'):
            # собираем данные в серию, где индексом будут некие ключевые параметры
            # а значениями - сумма по этим параметрам
            if item in result_series.index:
                result_series[item] += agg_parametr
            else:
                result_series[item] = agg_parametr
    # возвращаем итоговую серию данных
    return result_series.sort_values(ascending=sort_direction).head(top)
  

# создаем и заполняем новую колонку "Прибыльность"
data['profit'] = data.revenue - data.budget

# создаем и заполняем новую колонку "год выпуска фильма"
data['release_year'] = data.release_date.apply(year_detect)

# создаем и заполняем новую колонку "месяц выпуска фильма"
data['release_month'] = data.release_date.apply(month_detect)

# создаем и заполняем новую колонку "количество слов в обзоре фильма"
# функция поиска всех вхождений слов возвращает массив совпадений, его длина и есть количество слов
# словом считает все от одного символа и длинее
data['word_count_overview'] = data.overview.apply(lambda s: len(re.findall(r'\w+', s.lower())))


In [1731]:
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,word_count_overview
121,tt2911666,20000000,78739897,John Wick,Keanu Reeves|Michael Nyqvist|Alfie Allen|Wille...,Chad Stahelski|David Leitch,Don't set him off.,"After the sudden death of his beloved wife, Jo...",101,Action|Thriller,Thunder Road Pictures|Warner Bros.|87Eleven|De...,10/22/2014,7.0,2014,58739897,10,81
322,tt0457400,100000000,68688831,Land of the Lost,Will Ferrell|Anna Friel|Danny McBride|Jorma Ta...,Brad Silberling,Right place. Wrong time.,"On his latest expedition, Dr. Rick Marshall is...",102,Adventure|Comedy|Science Fiction,Universal Pictures|Mosaic Media Group|Relativi...,6/5/2009,5.4,2009,-31311169,6,48
558,tt0179626,60000000,56359980,15 Minutes,Robert De Niro|Charlize Theron|Edward Burns|Ke...,John Herzfeld,America Likes to Watch,When Eastern European criminals Oleg and Emil ...,120,Action|Crime|Thriller,New Line Cinema|Industry Entertainment|Tribeca...,3/1/2001,5.7,2001,-3640020,3,97
167,tt2975578,9000000,108782847,The Purge: Anarchy,Frank Grillo|Carmen Ejogo|Zach Gilford|Kiele S...,James DeMonaco,"Welcome to America, where one night a year, al...",Three groups of people are trying to survive P...,104,Horror|Thriller,Universal Pictures|Why Not Productions|Platinu...,7/17/2014,6.6,2014,99782847,7,30
610,tt0800080,150000000,163712074,The Incredible Hulk,Edward Norton|Liv Tyler|Tim Roth|William Hurt|...,Louis Leterrier,You'll like him when he's angry.,Scientist Bruce Banner scours the planet for a...,114,Science Fiction|Action|Adventure,Universal Pictures|Marvel Studios|Valhalla Mot...,6/12/2008,6.1,2008,13712074,6,47


 ####   Column            
---  ------               
 0.   imdb_id              
 1.   budget               
 2.   revenue              
 3.   original_title       
 4.   cast -- мультиполе, разделитель "|"
 5.   director -- мультиполе, разделитель "|"
 6.   tagline              
 7.   overview             
 8.   runtime              
 9.   genres -- мультиполе, разделитель "|"
 10.  production_companies -- мультиполе, разделитель "|"
 11.  release_date         
 12.  vote_average         
 13.  release_year         
 14.  profit
 15.  release_month
 16.  word_count_overiew

# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [1668]:
# получаем максимальный бюджет и фильтруем все фильмы с бюджетом равным максимальному
movie_name = data[data.budget == data.budget.max()]

# формируем и выдаем ответ в стиле платформы 
preparation_answer(movie_name)



Pirates of the Caribbean: On Stranger Tides (tt1298650)


In [1669]:
# +
answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

ВАРИАНТ 2

In [1670]:
# ранжируем колонку с бюджетами фильма, последнее значение будет максимальным
max_budget = data.budget.sort_values().tail(1)

# берем значение индекса у максмального значения и получаем название фильма
data.loc[max_budget.index[0]].original_title

'Pirates of the Caribbean: On Stranger Tides'

# 2. Какой из фильмов самый длительный (в минутах)?

In [1671]:
# получаем максимальную длительность и фильтруем все фильмы с длительностью равной максимальной
movie_name = data[data.runtime == data.runtime.max()]

preparation_answer(movie_name)


Gods and Generals (tt0279111)


In [1672]:
# +
answers['2'] = 'Gods and Generals (tt0279111)'

# 3. Какой из фильмов самый короткий (в минутах)?





In [1673]:
# получаем минимальную длительность и фильтруем все фильмы с длительностью равной минимальной
movie_name = data[data.runtime == data.runtime.min()]

preparation_answer(movie_name)


Winnie the Pooh (tt1449283)


In [1674]:
# +
answers['3'] = 'Winnie the Pooh (tt1449283)'

# 4. Какова средняя длительность фильмов?


In [1675]:
# получаем среднее значение по столбцу "длительность" и округляем до целого
round(data.runtime.mean())

110

In [1676]:
# +
answers['4'] = 110

# 5. Каково медианное значение длительности фильмов? 

In [1677]:
# получаем медианное значение по столбцу "длительность" и округляем до целого
round(data.runtime.median())

107

In [1678]:
# +
answers['5'] = 107

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [1679]:
# лучше код получения столбца profit вынести в Предобработку что в начале

# получаем максимальное значение из столбца "прибыль", получаем фильмы у которых прибыль максимальна
movie_name = data[data.profit == data.profit.max()]

preparation_answer(movie_name)

Avatar (tt0499549)


In [1680]:
# +
answers['6'] = 'Avatar (tt0499549)'

# 7. Какой фильм самый убыточный? 

In [1681]:
# получаем минимальное значение из столбца "прибыль", получаем фильмы у которых прибыль равна минимальной
movie_name = data[data.profit == data.profit.min()]

preparation_answer(movie_name)

The Lone Ranger (tt1210819)


In [1682]:
# +
answers['7'] = 'The Lone Ranger (tt1210819)'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [1683]:
# фильтруем датасет, выводим только те, у которых в колонке "прибыль" больше 0, определяем длину по вертикали,
# это и будет количеством фильмов
len(data[data.profit > 0])

1478

In [1684]:
# +
answers['8'] = 1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [1685]:
# фильтруем по столбцу "год выпуска", забирая только 2008 год, и получаем максимальное значение у этой выборки
# далее получаем фильмы у которых встречается максимальное значение в этой выборке
movie_name = data[data.profit == data.query('release_year == 2008').profit.max()]

preparation_answer(movie_name)

The Dark Knight (tt0468569)


In [1686]:
# +
answers['9'] = 'The Dark Knight (tt0468569)'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [1687]:
# фльтруем выборку по диапазону года выпуска, у этой выборке определеяем минимальный доход от фильма
# далее в этой выборке оставляем только фильмы с доходом равным минимальному
movie_name = data[data.profit == data.query('2012 <= release_year <= 2014').profit.min()]

preparation_answer(movie_name)

The Lone Ranger (tt1210819)


In [1688]:
# +
answers['10'] = 'The Lone Ranger (tt1210819)'

# 11. Какого жанра фильмов больше всего?

In [1689]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

# формируем серию где индексы - жанры, а значения - количество фильмов с таким жанром
# обрабатываем функцией, так как в поле жанр может быть несколько значений
series_aggregation_first_column(data.genres.value_counts(), 'sum', top=1)


Drama    782
dtype: int64

In [1690]:
# +
answers['11'] = 'Drama'

ВАРИАНТ 2

In [1691]:
# создаем пустую коллекцию
genre_counter = Counter()

# делаем выгрузку из дата фрейма и заполняем коллекцию значениями жанров
for genre_all, genre_count in data.genres.value_counts().items():
    for genre in genre_all.split('|'):
        genre_counter[genre] += genre_count

# получаем максимальное значения в словаре
max_count = max(genre_counter.values())

# ищем значение ключа, в котором находится жанр с максимальным количеством
for genre, item in genre_counter.items(): 
    if item == max_count: print(genre)


Drama


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [1692]:
# формируем серию из прибыльных фильмов, где индес - жанр, значение - количество фильмов
profitable_film_genre = data[data.profit > 0].groupby(['genres'])['profit'].count()

# обрабатываем функцией, так как в поле "жанр" может быть несколько значений
series_aggregation_first_column(profitable_film_genre, top=1)


Drama    560
dtype: int64

In [1694]:
# +
answers['12'] = 'Drama'

# 13. У какого режиссера самые большие суммарные кассовые сбооры?

In [1695]:
# формируем серию - группируем по полю директор и суммируя кассовые сборы
director_revenue = data.groupby(['director'])['revenue'].sum()

# обрабатываем функцией, так как в поле "директор" может быть несколько значений
series_aggregation_first_column(director_revenue, top=1)

Peter Jackson    6490593685
dtype: int64

In [1696]:
# +
answers['13'] = 'Peter Jackson'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [1697]:

director_genres = data[ ['director','genres'] ][data.genres.str.contains('Action', na=False)]

director_genres = director_genres.groupby(['director'])['genres'].count()

# обрабатываем функцией, так как в поле "директор " может быть несколько значений
series_aggregation_first_column(director_genres, 'sum', top=3)


Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
dtype: int64

In [1698]:
# +
answers['14'] = 'Robert Rodriguez'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [1699]:

top_actor_2012 = data.query('release_year == 2012').groupby(['cast'])['revenue'].sum()

# обрабатываем функцией, так как в поле "актер" может быть несколько значений
series_aggregation_first_column( top_actor_2012 ,'sum', top=1)


Chris Hemsworth    2027450773
dtype: int64

In [1700]:
# +
answers['15'] = 'Chris Hemsworth'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [1701]:
# высокобюджетные - фильмы, где бюджет выше среднего по данной выборке

top_actor = data[data.budget > data.budget.mean()].groupby(['cast'])['revenue'].count()

# обрабатываем функцией, так как в поле "актер" может быть несколько значений
series_aggregation_first_column(top_actor ,'sum', top=3)


Matt Damon        18
Adam Sandler      17
Angelina Jolie    16
dtype: int64

In [1702]:
# +
answers['16'] = 'Matt Damon'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [1703]:
favorite_genre = data[data.cast.str.contains('Nicolas Cage', na=False)].groupby(['genres'])['cast'].count()

# обрабатываем функцией, так как в поле "жанр" может быть несколько значений
series_aggregation_first_column( favorite_genre ,'sum', top=3)

Action      17
Thriller    15
Drama       12
dtype: int64

In [1704]:
answers['17'] = 'Action'

# 18. Самый убыточный фильм от Paramount Pictures

In [1705]:

movie_name = data[data.profit == data[data.production_companies.str.contains('Paramount Pictures')].profit.min()]

preparation_answer(movie_name)

K-19: The Widowmaker (tt0267626)


In [1706]:
answers['18'] = 'K-19: The Widowmaker (tt0267626)'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [1707]:
data.groupby(['release_year'])['revenue'].sum().sort_values().tail(1)

release_year
2015    25449202382
Name: revenue, dtype: int64

In [1708]:
answers['19'] = 2015

# 20. Какой самый прибыльный год для студии Warner Bros?

In [1709]:
wb_profit = data[data.production_companies.str.contains('Warner Bros', na=False)]

wb_profit.groupby(['release_year'])['profit'].sum().sort_values(ascending=False).head(1)

release_year
2014    2295464519
Name: profit, dtype: int64

In [1710]:
# +
answers['20'] = 2014

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [1711]:
# Вариант 1
data.release_month.value_counts().sort_values().tail(1)

9    227
Name: release_month, dtype: int64

In [1712]:
# Вариант 2
data.groupby(['release_month'])['imdb_id'].count().sort_values().tail(1)

release_month
9    227
Name: imdb_id, dtype: int64

In [1713]:
# +
answers['21'] = 'Сентябрь'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [1714]:
# 
len(data.query('release_month in [6, 7, 8]'))

450

In [1715]:
# +
answers['22'] = 450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [1716]:

winter_director = data.query('release_month in [1, 2, 12]').director.value_counts()

# обрабатываем функцией, так как в поле "директор" может быть несколько значений
series_aggregation_first_column(winter_director,'sum', top=3)

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
dtype: int64

In [1717]:
# +
answers['23'] = 'Peter Jackson'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [1718]:

max_len = data.original_title.str.len().max()

data[data.original_title.str.len() == max_len].production_companies.iloc[0].split('|')[1]



'Four By Two Productions'

In [1719]:
# +
answers['24'] = 'Four By Two Productions' 

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [1720]:
# 

word_count_overview_max = data[data.word_count_overview > data.word_count_overview.mean()].word_count_overview.max()

data[data.word_count_overview == word_count_overview_max].production_companies.iloc[0].split('|')[3]


'Midnight Picture Show'

In [1721]:
# +
answers['25'] = 'Midnight Picture Show'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [1722]:
# 
count_last_procent = round(0.01 * len(data))

# 
last_procent = data.vote_average.sort_values(ascending=False).head(count_last_procent).index

# 
top_movies = data.loc[last_procent][['original_title','vote_average']]
display(top_movies)

,original_title,vote_average
599,The Dark Knight,8.1
125,The Imitation Game,8.0
9,Inside Out,8.0
118,Interstellar,8.0
34,Room,8.0
1081,The Lord of the Rings: The Return of the King,7.9
138,The Grand Budapest Hotel,7.9
1183,The Wolf of Wall Street,7.9
370,Inception,7.9
119,Guardians of the Galaxy,7.9


In [1723]:
# Проверим, что из варианта №1 все 3 фильма находятся в в выборке 1% лучших фильмов

answer = ['Inside Out', 'The Dark Knight', '12 Years a Slave']

top_movies.query('original_title in @answer')


,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
1191,12 Years a Slave,7.9


In [1724]:
# +
answers['26'] = 'Inside Out, The Dark Knight, 12 Years a Slave'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [1725]:
actors_pair = Counter()

actors = data[data.cast.str.contains('|')].cast.unique()

for pair in actors:
    for actor in combinations(sorted(pair.split('|')), 2):
        # print(actor[0], actor[1])
        pair_key = actor[0] + ' & ' + actor[1]
        actors_pair[pair_key] += 1

count_colloboration_max = max(actors_pair.values())

# выведем топ 5 верхних значений по количеству совместных съемок
print(sorted(actors_pair.values(), reverse=True)[:5])

# выведем из словаря только пары актеров, с максимальным количеством совместных съемок
for pair, count_colloboration in actors_pair.items():
    if count_colloboration == count_colloboration_max: 
        print(pair)
    
# 


[8, 8, 8, 6, 6]
Daniel Radcliffe & Emma Watson
Daniel Radcliffe & Rupert Grint
Emma Watson & Rupert Grint


In [1726]:
# +
answers['27'] = 'Daniel Radcliffe & Rupert Grint'

ВАРИАНТ 2

In [1727]:
pair_actors = [  'Johnny Depp & Helena Bonham Carter',
                'Ben Stiller & Owen Wilson',
                'Vin Diesel & Paul Walker',
                'Adam Sandler & Kevin James',
                'Daniel Radcliffe & Rupert Grint']

actor_pair_work = pd.Series(dtype=object)

for pair_actor in pair_actors:
    actor1, actor2 =  pair_actor.split(' & ')
    pair_work_count = data[data.cast.str.contains(actor1) & data.cast.str.contains(actor2)].imdb_id.count()
    #print(pair_work_count, '|', pair_actor)
    actor_pair_work[pair_actor] = pair_work_count

actor_pair_work.sort_values(ascending=False).head(1).index[0]


'Daniel Radcliffe & Rupert Grint'

# Submission

In [1728]:
# в конце можно посмотреть свои ответы к каждому вопросу

# Вопрос № 1 ........ [ + ]
# Вопрос № 2 ........ [ + ]
# Вопрос № 3 ........ [ + ]
# Вопрос № 4 ........ [ + ]
# Вопрос № 5 ........ [ + ]
# Вопрос № 6 ........ [ + ]
# Вопрос № 7 ........ [ + ]
# Вопрос № 8 ........ [ + ]
# Вопрос № 9 ........ [ + ]
# Вопрос № 10 ....... [ + ]
# Вопрос № 11 ....... [ + ]
# Вопрос № 12 ....... [ + ]
# Вопрос № 13 ....... [ + ]
# Вопрос № 14 ....... [ + ]
# Вопрос № 15 ....... [ + ]
# Вопрос № 16 ....... [ + ]
# Вопрос № 17 ....... [ + ]
# Вопрос № 18 ....... [ + ]
# Вопрос № 19 ....... [ + ]
# Вопрос № 20 ....... [ + ]
# Вопрос № 21 ....... [ + ]
# Вопрос № 22 ....... [ + ]
# Вопрос № 23 ....... [ + ]
# Вопрос № 24 ....... [ + ]
# Вопрос № 25 ....... [ + ]
# Вопрос № 26 ....... [ + ]
# Вопрос № 27 ....... [ + ]

answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker (tt0267626)',
 '19': 2015,
 '20': 2014,
 '21': 'Сентябрь',
 '22': 450,
 '23': 'Peter Jackson',
 '24': 'Four By Two Productions',
 '25': 'Midnight Picture Show',
 '26': 'Inside Out, The Dark Knight, 12 Years a Slave',
 '27': 'Daniel Radcliffe & Rupert Grint'}

In [1729]:
# и убедиться что ни чего не пропустил)
len(answers)

27

### Спасибо за внимание!